In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de nulos

Una vez hecha la primera limpieza y transformación de datos que se guardó en el csv *`hr_limpio_con_nulos.csv`* procedemos a gestionar todos los datos nulos.

In [2]:
df_ejemplo = pd.read_csv('hr_limpio_con_nulos.csv', index_col=0)

In [3]:
df_ejemplo.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,yearbirth,remotework
0,51,No,NaN,684.0,NaN,6.0,University,NaN,162.0,1,male,51,3,Expert,resEArch DIREcToR,3,NaN,19537.0,6462,7,No,13,3.0,3,0,NaN,5,3.0,20,15,15,1972,Yes
1,52,No,NaN,699.0,NaN,1.0,Master Degree,Life Sciences,259.0,3,male,65,2,Expert,ManAGeR,3,NaN,19999.0,5678,0,NaN,14,3.0,1,1,34.0,5,3.0,33,11,9,1971,Yes


In [4]:
# como vamos a crear columnas nuevas, vamos a hacer una copia del DataFrame usando el método 'copy()'
df_copia = df_ejemplo.copy()
df_copia.head(1)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,yearbirth,remotework
0,51,No,NaN,684.0,NaN,6.0,University,NaN,162.0,1,male,51,3,Expert,resEArch DIREcToR,3,NaN,19537.0,6462,7,No,13,3.0,3,0,NaN,5,3.0,20,15,15,1972,Yes


_______

**Iterative Imputer:**

In [ ]:
def gestion_nulos_num ():
    imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)

    # ajustamos y tranformamos los datos
    imputer_iterative_imputado = imputer_iterative.fit_transform(df_copia[["dailyrate","totalworkingyears"]])

    # comprobamos que es lo que nos devuelve, que en este caso es un array también
    imputer_iterative_imputado

    # instanciamos la clase del KNNImputer
    imputer_knn = KNNImputer(n_neighbors = 5)

    # ajustamos y transformamos los datos
    imputer_knn_imputado = imputer_knn.fit_transform(df_copia[["dailyrate","totalworkingyears"]])

    # comprobamos que es lo que nos devuelve, que sigue siendo un array
    imputer_knn_imputado

    # por último nos queda añadir ese array al DataFrame como hemos hecho hasta ahora
    df_copia[["dailyrate_knn","totalworkingyears_knn"]] = imputer_knn_imputado

    # comprobamos los nulos
    print(f'Después del KNN tenemos: \n{df_copia[["dailyrate_knn","totalworkingyears_knn"]].isnull().sum()} nulos')

    df_copia.describe()[["dailyrate","dailyrate_iterative", "dailyrate_knn", "totalworkingyears", "totalworkingyears_iterative", "totalworkingyears_knn"]]

    #eliminamos las columnas que ya no nos interesan para guardar el DataFrame 
    df_copia.drop(["dailyrate", "totalworkingyears", "dailyrate_knn", "totalworkingyears_iterative" ], axis = 1, inplace = True)

    # ahora vamos a cambiar el nombre de las columnas que quedaron para que tengan el mismo nombre de origen
    nuevo_nombre = {"dailyrate_iterative": "dailyrate", "totalworkingyears_knn": "totalworkingyears"}
    df_copia.rename(columns = nuevo_nombre, inplace = True)

    median_worklifebalance = df_copia["worklifebalance"].median()
    print(f"La media de la columna 'worklifebalance' es: {round(median_worklifebalance, 2)}")

    # aplicamos el método 'fillna()' a la columna
    df_copia["worklifebalance"] = df_copia["worklifebalance"].fillna(median_worklifebalance)

    # comprobamos los nulos para la columna
    print(f"Después del 'fillna' tenemos {df_copia['worklifebalance'].isnull().sum()} nulos")
    
    # aplicamos el método 'fillna()' a la columna
    lista_columnas=["performancerating","monthlyincome", "employeenumber","distancefromhome"]

    df_copia[lista_columnas] = df_copia[lista_columnas].fillna("unknown")

    # comprobamos los nulos para la columna
    print(f"Después del 'fillna' tenemos {df_copia[lista_columnas].isnull().sum()} nulos")


_____
# Guardamos el DataFrame modificado

In [96]:

df_copia.to_csv("hr_limpio.csv")